#### Gain_ahead maior que 1 é igual ao ROC ou só o retorno shifado?

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import qgrid
import warnings
warnings.filterwarnings("ignore")
from app.tools.predictive_power import CPredictivePower
from app.market_data.mt5.get_data import CMT5MarketData
from app.core.calculator import Calculator

%matplotlib inline

In [6]:
mt5 = CMT5MarketData() 
stock_timeframes = [
    ("PETR4", "D1"),
]
collection = mt5.get_stocks(stock_timeframes)

In [7]:
ohlc = collection.collection[0]

In [10]:
from app.indicators.indicators_factory import IndicatorsFactory    
roc1, roc2 = IndicatorsFactory().build_from_dict([
    {"indicator": "roc","params": {"period": 1}},
    {"indicator": "roc","params": {"period": 2}}
])


In [18]:
calculator = Calculator(ohlc)
calculator.add_indicator(roc1)
calculator.add_indicator(roc2)

ppower = CPredictivePower(calculator.calc(), indicator)
ppower.estimate()


In [31]:
dataframe = ppower._dataframe.get()
dataframe["return_1"] = dataframe.close.pct_change() * 100
dataframe["return_2_shift"] = dataframe.return_1.shift(-2)
dataframe["return_2_rolling"] = dataframe.return_1.rolling(window=2).sum().shift(-2)
dataframe

,time,open,low,high,close,tick_volume,spread,real_volume,roc_1,roc_2,return_1,return_2_shift,return_2_rolling
0,2010-02-01,26.36,26.06,26.52,26.37,13168,1,15869300,NaN,NaN,NaN,-0.190694,-0.759522
1,2010-02-02,26.53,26.18,26.65,26.22,15765,1,15289600,-0.568828,NaN,-0.568828,-5.120367,-5.311061
2,2010-02-03,26.18,25.93,26.28,26.17,14085,1,16091200,-0.190694,-0.758438,-0.190694,-2.416432,-7.536799
3,2010-02-04,26.02,24.69,26.09,24.83,23120,1,24280300,-5.120367,-5.301297,-5.120367,0.701610,-1.714822
4,2010-02-05,24.45,23.62,24.81,24.23,27113,1,29033800,-2.416432,-7.413068,-2.416432,1.598361,2.299970
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,2020-05-21,19.50,19.07,19.77,19.19,109337,1,80920800,-0.569948,2.730193,-0.569948,4.338511,1.628766
2549,2020-05-22,18.80,18.35,18.90,18.67,78422,1,68841500,-2.709745,-3.264249,-2.709745,0.975359,5.313870
2550,2020-05-25,19.48,19.26,19.56,19.48,45778,1,37571100,4.338511,1.511204,4.338511,1.321810,2.297169
2551,2020-05-26,19.98,19.33,20.09,19.67,76325,1,68283100,0.975359,5.356186,0.975359,NaN,NaN


### Conclusão

Gain Ahead 2 estava errado, por representava o ganha de 1 barra de dois dias para frente o correto é a suma dos retornos de 2 dias
